In [57]:
#Authors: Dan Blum, Matt Greene, Bram Zilzter
#Dataset: https://www.kaggle.com/datasets/robikscube/flight-delay-dataset-20182022?select=readme.md

import pandas as pd
import os
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
# from sqlalchemy import create_engine, URL, MetaData
# import scipy.io

# test_file = '/app/res/test.csv'

# file = '../../res/Combined_Flights_2021.parquet'
# file = './res/Combined_Flights_2021.parquet'
test_file = '/app/raw/Flights_2018_1.csv'
# csv_file = pd.read_csv(file_2022_6, engine='pyarrow')
# print(list(csv_file.columns))
# for line in list(csv_file.columns):
#     print(line)
sample = pd.read_csv(test_file, engine='pyarrow')
print("I'm loaded")

ModuleNotFoundError: No module named 'matplotlib'

In [42]:
# sample = csv_file.sample(n = 100000)
# sample.to_csv("../../res/test.csv", index=False)
# sample.head(3).to_csv("res/sample.csv")

In [ ]:
# single_sample = csv_file(n = 100000)
# sample.to_csv("../../res/test.csv", index=False)

In [32]:
####### START OF EDA #######
#get all the flights in the first file that either depart from or arrive to boston
bos_sample = sample.query('Origin == "BOS" or Dest == "BOS"')
#print the number of flights from or to boston 
print(len(bos_sample.index))

18149


In [40]:
bos_sample.dtypes
###Flight date is actually in ISO 8601 format(YYYYMMDD), pandas doesn't recognize it, we can probably drop Year, Quarter, Month, DayofMonth, DayofWeek
bos_sample.FlightDate.dtype #need to give input format for ISO 8601 

dtype('O')

In [45]:
####### Check for duplicates #######
# bos_sample.groupby('Duplicate').count()
bos_sample['Duplicate'].value_counts()

N    18149
Name: Duplicate, dtype: int64

In [29]:
#split into departures from boston and arrive to boston
bos_dep_sample = bos_sample.query('Origin == "BOS"')

bos_arr_sample = bos_sample.query('Dest == "BOS"')

In [30]:
#what is the average delay time for the flights to or from boston

#departure delay: DepDelay - early departures as neg numbers
print(bos_sample['DepDelay'].mean())

#DepDel15(15 minutes or more as boolean)


#DepDelayMinutes delays greater than 15 minutes - early departures set to 0
print(bos_sample['DepDelayMinutes'].mean())

#Arrival Delay: ArrDelay - early has neg num
print(bos_sample['ArrDelay'].mean())

# ArrDelayMinutes - arrival times, early set to 0
print(bos_sample['ArrDelayMinutes'].mean())

#ArrDel15 - arrival times more than 15 minutes as boolean


16.100909793661177
19.646845454004875
9.584023351402871
19.72437004825162


In [27]:
#what is the average delay time for the flights from boston

#departure delay: DepDelay - early departures as neg numbers
print(bos_dep_sample['DepDelay'].mean())

#DepDel15(15 minutes or more as boolean)

#DepDelayMinutes delays greater than 15 minutes - early departures set to 0
print(bos_dep_sample['DepDelayMinutes'].mean())

#Arrival Delay: ArrDelay - early has neg num
print(bos_dep_sample['ArrDelay'].mean())

# ArrDelayMinutes - arrival times, early set to 0
print(bos_dep_sample['ArrDelayMinutes'].mean())

#ArrDel15 - arrival times more than 15 minutes as boolean

17.370546318289787
20.62565320665083
8.594404761904762
19.840595238095236


In [31]:
#what is the average delay time to the flights from boston

#departure delay: DepDelay - early departures as neg numbers
print(bos_arr_sample['DepDelay'].mean())

#DepDel15(15 minutes or more as boolean)


#DepDelayMinutes delays greater than 15 minutes - early departures set to 0
print(bos_arr_sample['DepDelayMinutes'].mean())

#Arrival Delay: ArrDelay - early has neg num
print(bos_arr_sample['ArrDelay'].mean())

# ArrDelayMinutes - arrival times, early set to 0
print(bos_arr_sample['ArrDelayMinutes'].mean())

#ArrDel15 - arrival times more than 15 minutes as boolean

14.827795641300465
18.66535667500298
10.575175867413854
19.607964707285085


In [46]:
bos = sample.query('Origin == "BOS" or Dest == "BOS"').to_numpy()
# sample['Origin']
# sample['OriginAirportSeqID']
print(bos[0:2])

[[2018 1 1 2 2 datetime.date(2018, 1, 2) 'DL' 'DL_CODESHARE' 19790 'DL'
  3300 '' nan '' nan '9E' 20363 '9E' 'N300PQ' 3300 11193 1119302 33105
  'CVG' 'Cincinnati, OH' 'KY' 21 'Kentucky' 52 10721 1072102 30721 'BOS'
  'Boston, MA' 'MA' 25 'Massachusetts' 13 910 1041.0 91.0 91.0 1.0 6.0
  '0900-0959' 15.0 1056.0 1237.0 10.0 1119 1247.0 88.0 88.0 1.0 5.0
  '1100-1159' 0.0 '' 0.0 129.0 126.0 101.0 1.0 752.0 4 88.0 0.0 0.0 0.0
  0.0 nan nan nan 0 nan nan nan nan '' nan nan nan nan nan nan '' '' nan
  nan nan nan nan nan '' None None None None None None None None None
  None None None None None None None None None None None None None None
  None 'N' None]
 [2018 1 1 3 3 datetime.date(2018, 1, 3) 'DL' 'DL_CODESHARE' 19790 'DL'
  3303 '' nan '' nan '9E' 20363 '9E' 'N905XJ' 3303 14492 1449202 34492
  'RDU' 'Raleigh/Durham, NC' 'NC' 37 'North Carolina' 36 10721 1072102
  30721 'BOS' 'Boston, MA' 'MA' 25 'Massachusetts' 13 1420 1423.0 3.0 3.0
  0.0 0.0 '1400-1459' 12.0 1435.0 1559.0 6.0 1619 160

In [49]:
bos_sample.Flights.unique()

array([1.])

In [58]:
sample_carr_delay = sample.query('CarrierDelay != 0.0')
sample_carr_delay['CarrierDelay'].value_counts()

3.0       1291
2.0       1290
1.0       1262
4.0       1246
5.0       1225
          ... 
777.0        1
822.0        1
529.0        1
1287.0       1
1118.0       1
Name: CarrierDelay, Length: 708, dtype: int64

In [60]:
sample_weath_delay = sample.query('WeatherDelay != 0.0')
sample_weath_delay['WeatherDelay'].value_counts()

4.0      149
7.0      144
6.0      138
9.0      134
15.0     132
        ... 
315.0      1
475.0      1
527.0      1
973.0      1
878.0      1
Name: WeatherDelay, Length: 476, dtype: int64

In [62]:
sample_nas_delay = sample.query('NASDelay != 0.0')
sample_nas_delay['NASDelay'].value_counts()

1.0       1616
15.0      1563
16.0      1430
2.0       1390
3.0       1291
          ... 
1180.0       1
492.0        1
239.0        1
743.0        1
368.0        1
Name: NASDelay, Length: 375, dtype: int64

In [64]:
sample_sec_delay = sample.query('SecurityDelay != 0.0')
sample_sec_delay['SecurityDelay'].value_counts()

16.0    11
4.0     10
10.0    10
18.0     9
15.0     8
        ..
76.0     1
56.0     1
66.0     1
96.0     1
39.0     1
Name: SecurityDelay, Length: 74, dtype: int64

In [65]:
sample_lac_delay = sample.query('LateAircraftDelay != 0.0')
sample_lac_delay['LateAircraftDelay'].value_counts()

15.0      945
16.0      938
17.0      928
18.0      842
19.0      770
         ... 
981.0       1
1379.0      1
656.0       1
746.0       1
280.0       1
Name: LateAircraftDelay, Length: 493, dtype: int64

In [ ]:
# file = open("/app/raw/Flights_2018_1.csv")
# print(file.readline())
# print(file.readline())
# bos = file[np.where(OriginAirportID == "BOS")]
# print(bos)

# for i in file:
#     print(i)




In [ ]:
# file_csv = '../../res/Combined_Flights_2021.csv'
# for line in list(pq_file.columns):
#     print(line)
# print(csv_file.columns)

# pd.read_csv(file_csv)

In [15]:
###### Dan's adding files to postgres ######
#testing the postgresql server
# url_object = URL.create(
#     "postgresql+psycopg2",
#     username="postgres",
#     password="postgres",
#     host="flights-db",
#     port="5432",
#     database="flights"
# )

# db = create_engine(url_object)k

# with db.connect() as conn:
#     s = 
    # for row in conn.execute()

conn_string = "host='flights-db' dbname='flights' user='postgres' password='postgres'"
with psycopg2.connect(conn_string) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM demo")
    records = cursor.fetchall()
    print(records)

[(1, 'testing'), (2, 'today')]
